In [ ]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

In [ ]:
def get_dataloaders(data_dir, batch_size=32):
    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform)
    val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform)
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, len(train_dataset.classes)

In [ ]:
data_dir = "/content/splits"
train_loader, val_loader, test_loader, num_classes = get_dataloaders(
    data_dir, batch_size=32
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

googlenet = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)
googlenet.fc = nn.Linear(1024, num_classes)
googlenet = googlenet.to(device)


Using device: cuda
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


100%|██████████| 49.7M/49.7M [00:00<00:00, 141MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(googlenet.parameters(), lr=1e-4)
num_epochs = 20

In [ ]:
# --- Training Loop ---
print("Starting training...")
for epoch in range(num_epochs):
    start_time = time.time()
    googlenet.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = googlenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    epoch_time = time.time() - start_time
    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_loss:.4f} - Time: {epoch_time:.2f}s")

print("Training finished.")

Starting training...


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/20] - Training Loss: 2.4874 - Time: 25.30s
Epoch [2/20] - Training Loss: 1.7704 - Time: 21.44s
Epoch [3/20] - Training Loss: 1.2727 - Time: 20.29s
Epoch [4/20] - Training Loss: 0.9243 - Time: 20.81s
Epoch [5/20] - Training Loss: 0.6443 - Time: 20.33s
Epoch [6/20] - Training Loss: 0.4326 - Time: 20.97s
Epoch [7/20] - Training Loss: 0.3017 - Time: 20.96s
Epoch [8/20] - Training Loss: 0.2041 - Time: 20.10s
Epoch [9/20] - Training Loss: 0.1474 - Time: 21.19s
Epoch [10/20] - Training Loss: 0.1167 - Time: 19.96s
Epoch [11/20] - Training Loss: 0.0931 - Time: 21.09s
Epoch [12/20] - Training Loss: 0.0749 - Time: 20.36s
Epoch [13/20] - Training Loss: 0.0622 - Time: 20.75s
Epoch [14/20] - Training Loss: 0.0623 - Time: 21.09s
Epoch [15/20] - Training Loss: 0.0555 - Time: 20.13s
Epoch [16/20] - Training Loss: 0.0551 - Time: 21.04s
Epoch [17/20] - Training Loss: 0.0503 - Time: 20.26s
Epoch [18/20] - Training Loss: 0.0417 - Time: 26.61s
Epoch [19/20] - Training Loss: 0.0437 - Time: 27.42s
Ep

In [ ]:
# --- Evaluation Loop ---
googlenet.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = googlenet(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [ ]:
# --- Metrics ---
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)

print("\nFinal Performance Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


Final Performance Metrics:
Accuracy : 0.7057
Precision: 0.7247
Recall   : 0.7057
F1 Score : 0.7064
